In [1]:
import torch
from transformers import AutoTokenizer
from gliner import GLiNER, GLiNERConfig

def setup_model():
    config = GLiNERConfig()
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    model = GLiNER.from_pretrained("gliner-community/gliner_small-v2.5")
    return model, tokenizer, config

# Initialize the model
model, tokenizer, config = setup_model()

2024-11-16 17:28:14.170656: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

pytorch_model.bin:   8%|7         | 52.4M/664M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Testing robustness to typos:
Evaluation Results: P: 71.43%	R: 78.95%	F1: 75.00%

F1 Score: 0.7500000000000001


In [2]:
def evaluate_test_case(test_data, entity_types, threshold=0.5):
    results, f1 = model.evaluate(test_data, flat_ner=True, threshold=threshold, entity_types=entity_types)
    print("Evaluation Results:", results)
    print("F1 Score:", f1)

In [20]:
text = "Ronaldo won the award. " * 500

In [6]:
# Test for robustness to typos
test_data_typo = [
    {"tokenized_text": ["Cristiano", "Roanldo", "has", "scored", "over", "800", "career", "goasl."],
     "ner": [(0, 1, "person"), (4, 6, "statistic")]},
    {"tokenized_text": ["Messi", "wsa", "bron", "on", "24", "June", "1987", "in", "Rosario."],
     "ner": [(0, 0, "person"), (4, 6, "date"), (8, 8, "location")]},
    {"tokenized_text": ["Al", "Nassr", "win", "teh", "Saudi", "Pro", "Leegue."],
     "ner": [(0, 1, "team"), (4, 6, "competition")]},
    {"tokenized_text": ["On", "Febrary", "1985,", "Ronaldo", "was", "bron", "in", "Funchal,", "Portugal."],
     "ner": [(1, 2, "date"), (3, 3, "person"), (7, 8, "location")]},
    {"tokenized_text": ["She", "is", "a", "develper", "at", "Microsfot."],
     "ner": [(5, 5, "company")]},
    {"tokenized_text": ["New", "Yrok", "is", "a", "busy", "city."],
     "ner": [(0, 1, "location")]},
    {"tokenized_text": ["Python", "progarmmers", "often", "use", "GitHub", "fr", "collaboration."],
     "ner": [(0, 0, "programming language"), (4, 4, "platform")]},
    {"tokenized_text": ["The", "Titanc", "sank", "in", "1912."],
     "ner": [(1, 1, "ship"), (4, 4, "date")]},
    {"tokenized_text": ["Elon", "Musk", "founder", "of", "Telsa", "is", "famus."],
     "ner": [(0, 1, "person"), (4, 4, "company")]},
    {"tokenized_text": ["Googl", "has", "introduced", "a", "new", "search", "feature."],
     "ner": [(0, 0, "company")]},
]

# Test for hyperparameter sensitivity with varied text inputs
test_data_hyperparameter = [
    {"tokenized_text": ["Ronaldo", "holds", "the", "record", "for", "most", "goals", "in", "the", "Champions", "League."],
     "ner": [(0, 0, "person"), (3, 4, "record"), (8, 10, "competition")]},
    {"tokenized_text": ["Python", "is", "a", "high-level", "programming", "language."],
     "ner": [(0, 0, "programming language")]},
    {"tokenized_text": ["The", "Eiffel", "Tower", "is", "in", "Paris."],
     "ner": [(1, 2, "landmark"), (5, 5, "location")]},
    {"tokenized_text": ["Apple", "announced", "the", "iPhone", "15."],
     "ner": [(0, 0, "company"), (3, 4, "product")]},
    {"tokenized_text": ["Microsoft", "launched", "Azure", "services", "in", "2023."],
     "ner": [(0, 0, "company"), (2, 2, "service"), (5, 5, "date")]},
    {"tokenized_text": ["Barack", "Obama", "served", "as", "the", "44th", "President", "of", "the", "United", "States."],
     "ner": [(0, 1, "person"), (9, 10, "location")]},
    {"tokenized_text": ["The", "Amazon", "rainforest", "is", "located", "in", "South", "America."],
     "ner": [(1, 1, "company"), (6, 7, "location")]},
    {"tokenized_text": ["Tesla", "and", "SpaceX", "are", "owned", "by", "Elon", "Musk."],
     "ner": [(0, 0, "company"), (2, 2, "company"), (6, 7, "person")]},
    {"tokenized_text": ["The", "Great", "Wall", "of", "China", "is", "a", "historic", "landmark."],
     "ner": [(1, 4, "landmark")]},
    {"tokenized_text": ["The", "Olympics", "is", "a", "global", "sports", "event."],
     "ner": [(1, 1, "competition")]},
]

# Test for edge cases
test_data_edge_cases = [
    {"tokenized_text": [], "ner": []},  # Empty text
    {"tokenized_text": ["Hello", "world!"], "ner": []},  # Non-entity text
    {"tokenized_text": ["Python"], "ner": []},  # Single non-entity word
    {"tokenized_text": ["!"], "ner": []},  # Single punctuation
    {"tokenized_text": ["123"], "ner": []},  # Numeric input
    {"tokenized_text": ["."], "ner": []},  # Single dot
    {"tokenized_text": [""], "ner": []},  # Empty string
    {"tokenized_text": ["Null"], "ner": []},  # Common ambiguous term
    {"tokenized_text": ["Unknown"], "ner": []},  # Generic word
    {"tokenized_text": ["Nothing"], "ner": []},  # Empty-seeming word
]

# Test for ambiguous contexts
test_data_ambiguous = [
    {"tokenized_text": ["Apple", "announced", "a", "new", "software", "update."],
     "ner": [(0, 0, "company")]},
    {"tokenized_text": ["Apple", "is", "a", "fruit", "and", "a", "tech", "giant."],
     "ner": [(0, 0, "fruit"), (5, 7, "company")]},
    {"tokenized_text": ["An", "apple", "a", "day", "keeps", "the", "doctor", "away."],
     "ner": [(1, 1, "fruit")]},
    {"tokenized_text": ["Windows", "is", "an", "operating", "system", "by", "Microsoft."],
     "ner": [(0, 0, "software"), (6, 6, "company")]},
    {"tokenized_text": ["Amazon", "is", "a", "river", "and", "also", "a", "company."],
     "ner": [(0, 0, "river"), (6, 6, "company")]},
    {"tokenized_text": ["Orange", "is", "a", "color", "and", "also", "a", "fruit."],
     "ner": [(0, 0, "color"), (6, 6, "fruit")]},
    {"tokenized_text": ["Tesla", "is", "known", "for", "electric", "vehicles."],
     "ner": [(0, 0, "company")]},
    {"tokenized_text": ["Jaguar", "is", "a", "car", "brand", "and", "an", "animal."],
     "ner": [(0, 0, "company"), (6, 6, "animal")]},
    {"tokenized_text": ["Python", "is", "a", "snake", "and", "a", "programming", "language."],
     "ner": [(0, 0, "animal"), (6, 7, "programming language")]},
    {"tokenized_text": ["Mercury", "is", "a", "planet", "and", "a", "chemical", "element."],
     "ner": [(0, 0, "planet"), (6, 7, "chemical element")]},
]
# Test for long documents
test_data_long_document = [
    {"tokenized_text": ["Ronaldo", "won", "the", "award."] * 500,
     "ner": [(0, 0, "person"), (2, 3, "award")]},
]

# Test for overlapping entities
test_data_overlapping_entities = [
    {"tokenized_text": ["Barack", "Obama", "visited", "the", "White", "House", "in", "Washington,",
                        "D.C."],
     "ner": [(0, 1, "person"), (4, 5, "landmark"), (7, 8, "location")]},
    {"tokenized_text": ["Amazon", "is", "a", "company", "and", "a", "river", "in", "Brazil."],
     "ner": [(0, 0, "company"), (6, 6, "river"), (8, 8, "location")]},
    {"tokenized_text": ["Apple", "is", "a", "fruit", "and", "also", "a", "company."],
     "ner": [(0, 0, "fruit"), (6, 6, "company")]},
    {"tokenized_text": ["Python", "is", "a", "language", "and", "a", "snake."],
     "ner": [(0, 0, "programming language"), (6, 6, "animal")]},
    {"tokenized_text": ["The", "Titanic", "sank", "in", "1912."],
     "ner": [(1, 1, "ship"), (4, 4, "date")]},
    {"tokenized_text": ["Elon", "Musk", "owns", "Tesla", "and", "SpaceX."],
     "ner": [(0, 1, "person"), (3, 3, "company"), (5, 5, "company")]},
    {"tokenized_text": ["Mercury", "is", "a", "planet", "and", "an", "element."],
     "ner": [(0, 0, "planet"), (6, 6, "chemical element")]},
    {"tokenized_text": ["Orange", "is", "a", "color", "and", "a", "fruit."],
     "ner": [(0, 0, "color"), (6, 6, "fruit")]},
    {"tokenized_text": ["Jaguar", "is", "a", "car", "brand", "and", "an", "animal."],
     "ner": [(0, 0, "company"), (6, 6, "animal")]},
    {"tokenized_text": ["Tesla", "is", "an", "electric", "vehicle", "company."],
     "ner": [(0, 0, "company")]},
]

# Test for diverse entities in long text
test_data_diverse_long_text = [
    {"tokenized_text": ["Barack", "Obama", "met", "Elon", "Musk", "at", "the", "White", "House", "on", "February", "15,", "2022."],
     "ner": [(0, 1, "person"), (3, 4, "person"), (7, 8, "landmark"), (10, 12, "date")]},
]

# Test for case sensitivity and punctuation
test_data_case_punctuation = [
    {"tokenized_text": ["Barack", "Obama", "is", "the", "44th", "President", "of", "the", "United", "States."],
     "ner": [(0, 1, "person"), (8, 9, "location")]},
    {"tokenized_text": ["Amazon", "offers", "fast", "delivery,", "especially", "for", "Prime", "members."],
     "ner": [(0, 0, "company"), (6, 6, "service")]},
    {"tokenized_text": ["Python", "is", "a", "language,", "not", "a", "snake!"],
     "ner": [(0, 0, "programming language")]},
    {"tokenized_text": ["Google's", "new", "AI", "tool,", "Bard,", "is", "a", "direct", "competitor", "to", "ChatGPT."],
     "ner": [(0, 0, "company"), (3, 4, "product")]},
    {"tokenized_text": ["Elon", "Musk,", "the", "CEO", "of", "Tesla,", "is", "a", "visionary."],
     "ner": [(0, 1, "person"), (5, 5, "company")]},
    {"tokenized_text": ["New", "York", "City", "is", "known", "as", "the", "\"Big", "Apple.\""],
     "ner": [(0, 2, "location")]},
    {"tokenized_text": ["The", "Eiffel", "Tower,", "located", "in", "Paris,", "is", "an", "iconic", "landmark."],
     "ner": [(1, 2, "landmark"), (4, 5, "location")]},
    {"tokenized_text": ["Dr.", "Jane", "Smith,", "a", "renowned", "scientist,", "works", "at", "NASA."],
     "ner": [(1, 2, "person"), (8, 8, "organization")]},
    {"tokenized_text": ["Mr.", "John", "Doe", "attended", "Harvard", "University."],
     "ner": [(1, 2, "person"), (4, 5, "organization")]},
    {"tokenized_text": ["The", "Great", "Gatsby,", "written", "by", "F.", "Scott", "Fitzgerald,", "is", "a", "classic."],
     "ner": [(1, 2, "book"), (5, 7, "author")]},
]


# Entity types for each test case
entity_types_typo = [
    "person", "statistic", "team", "competition", "date", "location", 
    "landmark", "company", "product", "ship", "platform", "programming language"
]

entity_types_hyperparameter = [
    "person", "record", "competition", "landmark", "company", "location", 
    "organization", "product", "service", "date"
]

entity_types_edge_cases = [
    "person", "location", "programming language", "company", "landmark", 
    "organization", "platform", "book", "record"
]

entity_types_ambiguous = [
    "company", "product", "fruit", "animal", "location", "organization", 
    "programming language", "software", "river", "color", "planet", "chemical element"
]

entity_types_long_document = [
    "person", "award", "programming language", "company", "landmark", 
    "competition", "service"
]

entity_types_overlapping = [
    "person", "company", "location", "landmark", "river", "animal", 
    "ship", "chemical element", "color", "planet"
]

entity_types_diverse_long_text = [
    "person", "team", "programming language", "platform", "landmark", 
    "competition", "company", "product", "service", "event", "date"
]

entity_types_case_punctuation = [
    "person", "location", "company", "service", "programming language", 
    "landmark", "organization", "book", "author"
]

# Run tests and display outputs
print("Testing robustness to typos:")
evaluate_test_case(test_data_typo, entity_types_typo)

print("\nTesting hyperparameter sensitivity (span length):")
evaluate_test_case(test_data_hyperparameter, entity_types_hyperparameter)

print("\nTesting edge cases (empty and single words):")
evaluate_test_case(test_data_edge_cases, entity_types_edge_cases)

print("\nTesting ambiguous contexts (e.g., Apple as company and fruit):")
evaluate_test_case(test_data_ambiguous, entity_types_ambiguous)

print("\nTesting long documents:")
evaluate_test_case(test_data_long_document, entity_types_long_document)

print("\nTesting overlapping entities:")
evaluate_test_case(test_data_overlapping_entities, entity_types_overlapping)

print("\nTesting diverse entities in long text:")
evaluate_test_case(test_data_diverse_long_text, entity_types_diverse_long_text)

print("\nTesting case sensitivity and punctuation:")
evaluate_test_case(test_data_case_punctuation, entity_types_case_punctuation)


Testing robustness to typos:
Evaluation Results: P: 62.50%	R: 78.95%	F1: 69.77%

F1 Score: 0.6976744186046512

Testing hyperparameter sensitivity (span length):
Evaluation Results: P: 75.00%	R: 75.00%	F1: 75.00%

F1 Score: 0.75

Testing edge cases (empty and single words):
Evaluation Results: P: 0.00%	R: 0.00%	F1: 0.00%

F1 Score: 0.0

Testing ambiguous contexts (e.g., Apple as company and fruit):
Evaluation Results: P: 55.56%	R: 58.82%	F1: 57.14%

F1 Score: 0.5714285714285715

Testing long documents:
Evaluation Results: P: 14.29%	R: 50.00%	F1: 22.22%

F1 Score: 0.22222222222222224

Testing overlapping entities:
Evaluation Results: P: 76.47%	R: 59.09%	F1: 66.67%

F1 Score: 0.6666666666666667

Testing diverse entities in long text:
Evaluation Results: P: 100.00%	R: 100.00%	F1: 100.00%

F1 Score: 1.0

Testing case sensitivity and punctuation:
Evaluation Results: P: 63.16%	R: 66.67%	F1: 64.86%

F1 Score: 0.6486486486486486


In [21]:
entities = model.predict_entities(text, ["person","award"])

for entity in entities:
    print(entity["text"], "=>", entity["label"])

Ronaldo => person
Ronaldo => person
Ronaldo => person
Ronaldo => person
Ronaldo won the award. Ronaldo won the award. Ronaldo => person
Ronaldo => person
Ronaldo => person
Ronaldo => person
Ronaldo => person
Ronaldo won the award. Ronaldo => person
Ronaldo => person
Ronaldo => person
Ronaldo => person
Ronaldo => person
Ronaldo won the award. Ronaldo won the award. Ronaldo => person
Ronaldo => person
Ronaldo => person
Ronaldo => person
Ronaldo => person
Ronaldo => person
Ronaldo won the award. Ronaldo won the award. Ronaldo => person
Ronaldo => person
Ronaldo won the award. Ronaldo won the award. Ronaldo => person
Ronaldo => person
Ronaldo => person
Ronaldo won the award. Ronaldo => person
Ronaldo won the award. Ronaldo => person
Ronaldo won the award. Ronaldo won the award. Ronaldo => person
Ronaldo => person
Ronaldo won the award. Ronaldo => person
Ronaldo => person
Ronaldo => person
Ronaldo won the award. Ronaldo won the award. Ronaldo => person
Ronaldo => person
Ronaldo => person
Ro